# Get data from the python client

## Relevant documents

- (Python Client Repo)[https://github.com/Green-Fusion/energy-management-backend/tree/main/python_client]
- (Swagger UI)[https://control.green-fusion.de/services/energy-management-backend/v1/api/doc#/]


## Inits

In [1]:
#imports
from energy_management_client import BackendPythonClient
import pandas as pd
import json
from datetime import datetime
import pprint  # For pretty-printing
import ast  
import os
import re
import numpy as np # Often used for NaN handling

In [2]:
# select your environment
environment="prod" # prod or dev

# Selector
if environment=="dev":
    backend_endpoint_path="https://dev.green-fusion.de/services/energy-management-backend/v1/api"
    realm_type="development"
elif environment=="prod":
    backend_endpoint_path="https://control.green-fusion.de/services/energy-management-backend/v1/api"
    realm_type="control"  
else:
    backend_endpoint_path=""
    realm_type=""
    print("select either prod or dev for the environment")

#test
#backend_endpoint_path, realm_type
if True:
    # pythpon client auth & login
    # init the client   
    client = BackendPythonClient(backend_endpoint=backend_endpoint_path,)
    
    #login
    client.login(
        auth_endpoint="https://auth.green-fusion.de",
        realm=realm_type,
        client_id="backend_python_client", # backend_python_client, datascience_development_api
        grant_type="device_code", #"client_credentials",  # or "password", "device_code"
    )

## Functions

### buildings

In [3]:
def get_buildings_df(client, customers):
    """
    Loop through all customers, fetch their buildings, and build a DataFrame where each row contains:
      - building_id, customerID (from customer object), address, city, atvise_display_name, base,
      - heatingType, heatingSurface, access_key, nickname, postal_code, gui_name,
      - coordinates, CustomerAlias, gfid,
      - LayoutID (extracted from modular_system)

    Usage:
        customers = client.customers.get_list_of_customers()
        df = get_buildings_df(client, customers)
        print(df.head())
    """
    try:
        data = []
        for c in customers:
            customer_id = c.id
            customer_name = getattr(c, 'name', None)
            client.configure(customer_id=customer_id)
            
            # Get buildings
            c_buildings = client.buildings.get_buildings()
            building_list = getattr(c_buildings, 'buildings', [])
            
            for b in building_list:
                building_id         = getattr(b, 'building_id', None)
                address             = getattr(b, 'address', None)
                city                = getattr(b, 'city', None)
                atvise_display_name = getattr(b, 'atvise_display_name', None)
                base                = getattr(b, 'base', None)
                heatingType         = getattr(b, 'heatingType', None)
                heatingSurface      = getattr(b, 'heatingSurface', None)
                access_key          = getattr(b, 'access_key', None)
                nickname            = getattr(b, 'nickname', None)
                postal_code         = getattr(b, 'postal_code', None)
                gui_name            = getattr(b, 'gui_name', None)
                coordinates         = getattr(b, 'coordinates', None)
                CustomerAlias       = getattr(b, 'CustomerAlias', None)
                gfid                = getattr(b, 'gfid', None)
                
                # Parse modular_system
                modular_system_str = getattr(b, 'modular_system', None)
                LayoutID = None
                if modular_system_str:
                    try:
                        modular_system = json.loads(modular_system_str)
                        LayoutID = modular_system.get('layoutID', None)
                    except json.JSONDecodeError:
                        LayoutID = None

                data.append({
                    "building_id": building_id,
                    "customerID": customer_id,  # <-- corrected here
                    "customer_name": customer_name,
                    "address": address,
                    "city": city,
                    #"atvise_display_name": atvise_display_name,
                    #"base": base,
                    #"heatingType": heatingType,
                    #"heatingSurface": heatingSurface,
                    #"access_key": access_key,
                    #"nickname": nickname,
                    "postal_code": postal_code,
                    #"gui_name": gui_name,
                    "coordinates": coordinates,
                    #"CustomerAlias": CustomerAlias,
                    "gfid": gfid,
                    "modular_system": modular_system,  
                    "LayoutID": LayoutID
                })

        return pd.DataFrame(data)

    except Exception as e:
        print("API Call Failed:", e)
        return pd.DataFrame()

### devices

In [4]:
def get_devices(client):
    """
    usage:
    list_device=get_devices(client)
    list(list_device)
    """
    devices_list=client.devices.get_devices()
    devices_list=list(devices_list)[0][1]
    device_dicts = [device.__dict__ for device in devices_list]
    df_device = pd.DataFrame(device_dicts)
    return df_device

### Sensors  

In [5]:
def get_sensors(building_id):
    sensors = client.buildings.get_list_of_sensors_by_building_id(building_id)
    
    sensors_infos = {
        f"{s.acronym}({s.hydraulic_location_index})": f"id : {s.id} gfid : {s.gf_standard_id} sub_system_id : {s.sub_system_id} unit : {s.unit} source : {s.source}"
        for s in sensors
    }
    return sensors_infos

In [6]:
#get_sensors(2382)

#### count

In [7]:
def get_t_sensor_count(building_id):
    try:
        sensors = client.buildings.get_list_of_sensors_by_building_id(building_id)
        count = sum(
            1 for s in sensors
            if s.unit == '°C'
            and s.source == 'GREENBOX_MQTT'
            and (s.long_name is None or 'Wärmemengenzähler' not in s.long_name)
        )
        return count
    except Exception as e:
        # Optionally log the error
        return None


In [8]:
#get_t_sensor_count(1750)

In [9]:
#client.buildings.get_list_of_sensors_by_building_id(2382)

#### missing sensors ww

In [10]:
#def find_missing_sensors_by_gfid(row):
def find_missing_sensors_by_gfid(row):
    """
    Finds missing warm water sensors and includes all relevant subcomponent IDs in the result.

    Args:
        row (pd.Series): A single row from the DataFrame.

    Returns:
        dict: A dictionary with missing GFIDs and the IDs of the warm water subsystems checked.
              Returns None if no sensors are missing or no warm water systems exist.
    """
    REQUIRED_WW_SENSORS_gfid = {118, 119, 123, 124}
    sub_components = row['sub_components']
    sensors_data = row['sensors']

    if not isinstance(sub_components, dict) or not isinstance(sensors_data, dict):
        return None

    # 1. Find all subsystem_ids for 'WARM_WATER'
    ww_subsystem_ids = {
        int(match.group(1))
        for comp_details in sub_components.values()
        if "type : WARM_WATER" in comp_details and (match := re.search(r"id : (\d+)", comp_details))
    }

    if not ww_subsystem_ids:
        return None

    # 2. Find GFIDs of sensors belonging to ANY of the warm water subsystems
    existing_sensors_gfids = {
        int(gfid_match.group(1))
        for sensor_details in sensors_data.values()
        if (sub_id_match := re.search(r"sub_system_id : (\d+)", sensor_details))
        and int(sub_id_match.group(1)) in ww_subsystem_ids
        and (gfid_match := re.search(r"gfid : (\d+)", sensor_details))
    }

    # 3. Compare the set of existing GFIDs with the required set
    missing_gfids = sorted(list(REQUIRED_WW_SENSORS_gfid - existing_sensors_gfids))

    if not missing_gfids:
        return None

    # Return a dictionary containing both the missing GFIDs and the list of WW subsystem IDs
    return {
        'missing_gfids': missing_gfids,
        'ww_subsystem_ids': sorted(list(ww_subsystem_ids))
    }

### sub component

In [11]:
def get_sub_component(building_id):
    subsystems = client.buildings.get_sub_systems_by_building_id(building_id)
    
    subsystems_infos = {
        f"{s.acronym}({s.hydraulic_location_index})": f"id : {s.id} category : {s.category} type : {s.type} sub_type : {s.sub_type}"
        for s in subsystems
    }
    return subsystems_infos

In [12]:
#client.buildings.get_sub_systems_by_building_id(2382) 


In [13]:
#get_sub_component(2382)

## get_buildings_df

In [14]:
# Get customers list (list of dicts)
customers = client.customers.get_list_of_customers()
#build df
df_buildings = get_buildings_df(client, customers)
#test
#df_buildings.head()

## get sub component 

In [15]:
df_buildings['sub_components'] = df_buildings['building_id'].apply(get_sub_component)

In [16]:
#df_buildings[['building_id','LayoutID','sub_components']].head()

In [17]:
#df_buildings.loc[df_buildings['building_id'] == 765, 'sub_components']

## get sub sensors 

In [18]:
df_buildings['sensors'] = df_buildings['building_id'].apply(get_sensors)

## get sensor count

In [19]:
df_buildings['t_sensor_count']=0
#df_buildings['t_sensor_count'] = df_buildings['building_id'].apply(get_t_sensor_count)

In [20]:
#df_buildings.count()

## get devices

In [21]:
df_devices=get_devices(client)
df_devices = df_devices.rename(columns={"type": "device_type"})

In [22]:
#df_devices.head()

In [23]:
#df_devices.count()

In [24]:
df_devices[['device_type','building_id']].groupby('device_type').count()

,building_id
device_type,
ECR_LW300,129
RUT956,988
RevPiConnectSE,1
RevPiCore32SE,3
WAGOPFC200,1


## Merge Dfs

In [25]:
# Merge the two DataFrames on building_id
df_buildings = df_buildings.merge(
    df_devices[['building_id', 'device_type', 'device_id']],
    on='building_id',
    how='left'  # ensures all rows from df_buildings are kept
)

In [26]:
#df_buildings.head()

In [27]:
#df_buildings.count()

## Get missing_sensors df

In [28]:
df_buildings['missing_WW_sensors'] = df_buildings.apply(find_missing_sensors_by_gfid, axis=1)
df_missing_WW_sensors=df_buildings[df_buildings['missing_WW_sensors'].notna()]

In [29]:
df_missing_WW_sensors=df_missing_WW_sensors[['building_id',
               'customerID',
               'customer_name',
               'address',
               'postal_code',
               'city',
               'device_type',
               'device_id',
               'sub_components',
               'sensors',
               'missing_WW_sensors',
               'gfid']]

### test

In [37]:
bid=420 

In [38]:
df_buildings.loc[df_buildings['building_id'] == bid, 'sub_components'].item()

{'global(1)': 'id : 988 category : NOT_SPECIFIED type : GLOBAL sub_type : NOT_SPECIFIED',
 'hp(n)(1)': 'id : 989 category : PRODUCER type : HEAT_PUMP sub_type : NOT_SPECIFIED',
 'hc(n)(1)': 'id : 990 category : CONSUMER type : HEATING_CIRCUIT sub_type : NOT_SPECIFIED',
 'hw(n)(1)': 'id : 991 category : CONSUMER type : WARM_WATER sub_type : FRESH_WATER_STATION'}

In [39]:
df_buildings.loc[df_buildings['building_id'] == bid, 'missing_WW_sensors'].item()

In [40]:
df_buildings.loc[df_buildings['building_id'] == bid, 'sensors'].item()

{'t_hp(n)_secflow(1)': 'id : 7728 gfid : 369 sub_system_id : 989 unit : °C source : GREENBOX_MQTT',
 'e_g_bldg_y(None)': 'id : 19387 gfid : 460 sub_system_id : None unit : kWh source : MANUALLY_ENTERED',
 't_g_out_api(1)': 'id : 7726 gfid : 434 sub_system_id : 988 unit : °C source : OPEN_WEATHER_API',
 't_hp(n)_primflow(1)': 'id : 9510 gfid : 403 sub_system_id : 989 unit : °C source : GREENBOX_MQTT',
 't_hp(n)_primreturn(1)': 'id : 9509 gfid : 408 sub_system_id : 989 unit : °C source : GREENBOX_MQTT',
 't_hp(n)_secreturn(1)': 'id : 7732 gfid : 371 sub_system_id : 989 unit : °C source : GREENBOX_MQTT',
 't_hw(n)_circ(1)': 'id : 7731 gfid : 118 sub_system_id : 991 unit : °C source : GREENBOX_MQTT',
 't_hw(n)_pwh(1)': 'id : 7730 gfid : 119 sub_system_id : 991 unit : °C source : GREENBOX_MQTT',
 't_hc(n)_flow(1)': 'id : 7733 gfid : 112 sub_system_id : 990 unit : °C source : GREENBOX_MQTT',
 't_hw(n)_tank(1)': 'id : 7735 gfid : 124 sub_system_id : 991 unit : °C source : GREENBOX_MQTT',
 't_

## Exports df to csv

In [36]:
# get_buildings_df
if True:
    today_str = datetime.today().strftime('%Y-%m-%d')
    filename = f"Buildings_infos_{today_str}.csv"
    filename2 = f"Building_missing_ww_sensors{today_str}.csv"
    filepath = os.path.join('OUTs', filename)
    filepath2 = os.path.join('OUTs', filename2)
    df_buildings[['building_id',
               'customerID',
               'customer_name',
               'address',
               'postal_code',
               'city',
               'device_type',
               'device_id',
               'LayoutID',
               'sub_components',
               'modular_system',
               't_sensor_count',
               'sensors',
               'missing_WW_sensors',
               'gfid']].to_csv(filepath, index=False)
    df_missing_WW_sensors[['building_id',
               'customerID',
               'customer_name',
               'address',
               'postal_code',
               'city',
               'device_type',
               'device_id',
               'sub_components',
               'sensors',
               'missing_WW_sensors']].to_csv(filepath2, index=False)